# <font color=blue>OmegAlpes Tutorial 3 - Grid Cost and Storage
`Prerequisite :` <a href="./OmegAlpes_Tutorial_1.ipynb">Tutorial 1,</a>
    <a href="./OmegAlpes_Tutorial_2.ipynb">Tutorial 2</a><br><br><br>
<font color=red>There are some parts in this tutorial that has already been covered in previous Tutorial 1 and will not be covered here. Energy storage part is covered in Tutorial 2</font>
    
    In this tutorial, we will learn to add production cost components and use it as optimisation objective. With these feature, we will explore the suggestion to consider cost of production in reducing his electricity bill.
    
   `Learning outcomes :`
>1. New vector in the model that is non-energy : production cost
2. New attribute for Variable Units
3. New kind of optimisation objective
   
    At the end of this tutorial, you will be able to recommend which grid to choose and when to charge-discharge storage

<img src="./images/3-combine.jpg" width="600" height="300" >

In [ ]:
from Python_Scripts.Omegalpes_tutorial_3 import *

    Similar to previous Tutorial 1. Try to have similar steps to have a good programming structure.

## Create time unit   

In [ ]:
time = TimeUnit(periods=24*1, dt=1)

    where :
    dt = 1 hour / 60 minutes since all of the production and consumption data is in 1-hour timestep
    period = 24 hours. You can have longer period if the data is available.
    
`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.time">time module</a>

## Create empty model
    

In [ ]:
model = OptimisationModel(time=time, name='tutorial_3')

  
`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">model module</a>

## Create energy unit

### Create fixed energy units

In [ ]:
consumption_profile = open("data/house tot con 24 kwh.csv", "r")

house_cons_file = [p for p in map(float, consumption_profile)]

house_consumption = FixedConsumptionUnit(time, 'johns_consumption', p=house_cons_file, energy_type=elec)

In [ ]:
pv_profile = open("data/pv prod 24 kwh.csv", "r")

pv_production_daily = [p for p in map(float, pv_profile)]

pv_production = FixedProductionUnit(time, name='pv_production', p=pv_production_daily ,energy_type=elec)

    Now, we have created 2 fixed energy units
<img src="./images/1-fixed_units.JPG" width="550" height="150">

### Create variable energy units
     As mentioned at the beginning of this tutorial, we are going to give recommendation base on operating cost. So, we will add this attribute into these Variable units.
     
     First, let's start with creating grid export.

In [ ]:
#Hourly operating cost data
export_cost = [50, 50, 50, 50, 50,50, 50, 50, 50, 50, 50, 80, 80, 20, 30,50, 50, 50, 50, 50,50, 50, 50, 50]
                    
grid_export = VariableConsumptionUnit(time, 'grid_export', energy_type=elec, p_min=0,operating_cost=export_cost)

    Next, we create grid import where there are 2 options to choose (grid a and grid b).

In [ ]:
# Hourly operating costs for the first production unit
op_cost_a = [41.1, 41.295, 43.125, 51.96, 58.275,62.955, 58.08, 57.705, 59.94, 52.8, 53.865,
             46.545, 41.4, 39, 36.87,36.6, 39.15, 43.71, 45.195, 47.04, 44.28, 39.975, 34.815, 28.38]   
                
# Hourly operating costs for the second production unit
op_cost_b = [58.82, 58.23, 51.95, 47.27, 45.49,44.5, 44.5, 44.72, 44.22, 42.06,45.7,
             47.91, 49.57, 48.69, 46.91, 46.51, 46.52, 51.59, 59.07, 62.1, 56.26, 55, 56.02, 52]
      
grid_production_a = VariableProductionUnit(time=time,
                                          name='grid_production_A',
                                          energy_type=elec,
                                          operating_cost=op_cost_a)


grid_production_b = VariableProductionUnit(time=time,
                                           name='grid_production_B',
                                           energy_type=elec,
                                           operating_cost=op_cost_b)

`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.energy_nodes">energy nodes module</a>

    Now, we have three 3 energy units to add to our model.
<img src="./images/3-variable_units.JPG" width="550" height="150">
<br>

### Create storage units

In [ ]:
storage = StorageUnit(time, name='storage', pc_max=1, pd_max=1, soc_min=0.1, soc_max=0.9, self_disch=0.01, ef_is_e0=True)

`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.storage_units">storage units module</a>

    Now, we add storage unit component in our model
    
<img src="./images/3-storage_unit.JPG" width="550" height="150">
<br>

## Define model objective
    The objective will be different since we want to compare cost. So, we will be using minimize_operating_cost() objective. Now, how should we write it ?

In [ ]:
%run quiz_tutorial31

In [ ]:
#write 2 model objectives line here

`learn more optimization objective functions for energy units:`<a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.production_units">production units module</a> & <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.consumption_units"> consumption units module</a>

## Create energy nodes

In [ ]:
elec_node = EnergyNode(time, name="electrical_node", energy_type=elec)

    Then, we connect energy units to the energy node we create, both grid A and B included

In [ ]:
elec_node.connect_units(pv_production, house_consumption, grid_production_a, grid_production_b, grid_export, storage)

    Great! Now, we have created our energy system
<img src="./images/3-node.JPG" width="700" height="300">



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.energy_nodes">energy nodes module</a>

## Add the energy nodes to the optimisation model
    Remember to check that the timeunit is the same for the model and all the units

In [ ]:
model.add_nodes(elec_node)



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">optimisation model</a>

## Launch the optimisation

In [ ]:
model.solve_and_update()



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">optimisation model</a>

## Plot result

In [ ]:
plot_node_energetic_flows(elec_node)

`learn more plotting functions:` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.utils.plots">plots module</a>

## Result processing
    Showing the result

In [ ]:
plt.show()

    After Tutorial 2, you should have acquired the skill of sum_quantities_in_quantity() function. Let's use your new skill to solve this challenge!

In [1]:
%run quiz_tutorial32

In [ ]:
#work on your challenge here

`learn more plotting functions:` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.utils.plots">plots module</a>   


## Discussion
    What do you think about the result? By adding new kind of objectives, OmegAlpes considers not only energy surplus but also the cost in the event of charging battery. This is very handful when you have multiple objectives to achieve.
    
    Do you know how much import has been reduced? Try finding out yourself below and compare with other results! How about combining all the level 2 skills you get and propose a better recommendation for John!

    Let us return to the dashboard so we can progress more in finding optimal solution for John with the other level 2 Tuto-rial. Don't forget to collect 2 level 2 magic words in this tutorial by finishing tasks in order to unlock stuffs!
    
`[Link]` <a href="./OmegAlpes_tutorial.ipynb">Dashboard</a><br><br>